In [1]:
import os
import gc
import cv2
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from lightly.loss import NTXentLoss

# Utils
from tqdm.auto import tqdm

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorch_toolbelt import losses as L

## using gpu:1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

In [2]:
class SSL_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        self.model = timm.create_model(model_name, 
                                       pretrained=True, 
                                       num_classes=cls, 
                                       drop_rate= CFG['drop_out'], 
                                       drop_path_rate= CFG['drop_path'])
        
    def forward(self, image):
        x = self.model(image)
        return x

In [3]:
def get_train_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit= 10,
                                        interpolation=cv2.INTER_LINEAR, border_mode=0, p=0.7),
        ToTensorV2(p=1.0),
    ])


def get_test_transform(img_size):
    return A.Compose([
        A.SmallestMaxSize(max_size=img_size, interpolation=3, p=1),
        ToTensorV2(p=1.0),
    ])

In [4]:
class Customize_Dataset(Dataset):
    def __init__(self, df, train_transforms, test_transforms):
        self.df = df
        self.train_transforms = train_transforms
        self.test_transforms = test_transforms
    
    def __getitem__(self, index):
        data = self.df.loc[index]
        img = cv2.imread(data['image_path'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        ori_img = self.test_transforms(image=img)["image"]
        aug_img = self.train_transforms(image=img)["image"]
            
        return {
            'origin_image': torch.tensor(ori_img/255, dtype=torch.float32),
            'augment_image': torch.tensor(aug_img/255, dtype=torch.float32),
        }
    
    def __len__(self):
        return len(self.df)

In [5]:
class Customize_loss(nn.Module):
    def  __init__(self):
        super().__init__()
        self.NTXloss= NTXentLoss()
    
    def forward(self, y_pred, y_true):
        loss= self.NTXloss(y_pred, y_true)
        return loss

In [6]:
def train_epoch(dataloader, model, criterion, optimizer):
    scaler= amp.GradScaler()
    model.train()

    ep_loss= []
    for i, data in enumerate(tqdm(dataloader)):

        imgs_ori= data['origin_image'].to('cuda')
        imgs_aug= data['augment_image'].to('cuda')
        
        with amp.autocast():
            embd_1= model(imgs_ori)
            embd_2= model(imgs_aug)
            
            loss= criterion(embd_1, embd_2)
            ep_loss.append(loss.item())
            loss/= CFG['gradient_accumulation']
            scaler.scale(loss).backward()
            
            if (i+1) % CFG['gradient_accumulation']== 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                
    return np.mean(ep_loss)

# CFG

In [7]:
CFG= {
    'fold': 0,
    'epoch': 30,
    'model_name': 'tf_efficientnet_b0_ns',
    
    'img_size': 128,
    'batch_size': 64,
    'gradient_accumulation': 1,
    'gradient_checkpoint': False,
    'drop_out': 0.3,
    'drop_path': 0.2,
    
    'lr': 3e-4,
    'weight_decay': 3e-4,
    
    'num_classes': 264,
    'load_model': False, 
    'save_model': './train_model_copy'
}

# Prepare Dataset

In [8]:
df= pd.read_csv('data/train.csv')
train_dataset= df[df['fold']!=CFG['fold']].reset_index(drop=True)
print(f'train dataset: {len(train_dataset)}')

train_dataset= Customize_Dataset(train_dataset, get_train_transform(CFG['img_size']), get_test_transform(CFG['img_size']))
train_loader= DataLoader(train_dataset, batch_size= CFG['batch_size'], shuffle=True, num_workers=0)
df.head()

train dataset: 104997


,image_path,label,label_name,group,fold
0,Data/train_img\abethr1\XC128013._0.png,0,abethr1,XC128013.,1.0
1,Data/train_img\abethr1\XC128013._1.png,0,abethr1,XC128013.,1.0
2,Data/train_img\abethr1\XC128013._2.png,0,abethr1,XC128013.,1.0
3,Data/train_img\abethr1\XC128013._3.png,0,abethr1,XC128013.,1.0
4,Data/train_img\abethr1\XC128013._4.png,0,abethr1,XC128013.,1.0


# Train

In [9]:
## create model
if CFG['load_model']:
    print(f"load_model: {CFG['load_model']}")
    model= torch.load(CFG['load_model'], map_location= 'cuda')
else:
    model= SSL_Model(CFG['model_name'], CFG['num_classes'])
model.to('cuda')
    
## hyperparameter
criterion= Customize_loss()
optimizer= optim.AdamW(model.parameters(), lr= CFG['lr'], weight_decay= CFG['weight_decay'])

## start training
for ep in range(1, CFG['epoch']+1):
    print(f'\nep: {ep}')
    
    train_loss= train_epoch(train_loader, model, criterion, optimizer)
    print(f'train loss: {round(train_loss, 5)}')
        
    ## save model every epoch
    torch.save(model, f"{CFG['save_model']}/cv{CFG['fold']}_ep{ep}.pth")

C:\Users\User\anaconda3\envs\kaggle_smbcd\lib\site-packages\timm\models\_factory.py:97: UserWarning: Mapping deprecated model tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  **kwargs,



ep: 1


  0%|          | 0/1641 [00:00<?, ?it/s]

C:\Users\User\anaconda3\envs\kaggle_smbcd\lib\site-packages\ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\User\anaconda3\envs\kaggle_smbcd\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


train loss: 3.12413

ep: 2


...

train loss: 2.92459

ep: 29


  0%|          | 0/1641 [00:00<?, ?it/s]

train loss: 2.92361

ep: 30


  0%|          | 0/1641 [00:00<?, ?it/s]

train loss: 2.92326
